In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")
import ase
import time
import glob
import ase.io as aio
import scipy.io as sio
import poremks
import poremks.grid_generator as gen
import poremks.porosity as pore
from poremks.helpers import write2vtk, sphere
import numpy as np
import ase.visualize as vis
import matplotlib.pyplot as plt
from toolz.curried import pipe, curry

## Load Structure of Interest

In [3]:
%%time
r_Ox = 1.35
r_Si = 1.35

len_pixel = 10


fname = "MFI.cif"

cif = fname.split("/")[-1][:-4]
rep = [1, 1, 1]
atom = pipe(fname, 
            lambda fname: aio.read(fname), 
            lambda x: x.repeat(rep))

print("No. of atoms : %d" % len(atom))
print(atom.get_cell().diagonal())
print("Orthogonal Unit Cell:", np.all(atom.get_cell_lengths_and_angles()[3:] == 90.0))

No. of atoms : 288
[20.09  19.738 13.142]
Orthogonal Unit Cell: True
CPU times: user 91.2 ms, sys: 193 µs, total: 91.4 ms
Wall time: 90.3 ms


## Generate Voxelized Representation of the Pore Structure

In [6]:
%%time
radii={"Si":r_Si, "O": r_Ox}
S, S_list, box_dim = gen.grid_maker(atom, len_pixel=10, radii=radii, full=False, fft=True)
print(S.shape)

(202, 198, 133)
CPU times: user 1.65 s, sys: 284 ms, total: 1.93 s
Wall time: 996 ms


In [5]:
write2vtk(S, "mfi_pore.vtk")
write2vtk(S_list[0], "mfi_o.vtk")
write2vtk(S_list[1], "mfi_si.vtk")

NameError: name 'S' is not defined

## Conventional Pore Metrics - PLD and LCD

In [ ]:
strt = time.time()
padval = ((1, 1), (1, 1), (0, 0)) 
S_dgrid = pipe(S,
               lambda s: np.pad(S, padval, 'constant', constant_values=0),
               lambda s: pore.dgrid(S, len_pixel=len_pixel))
end = time.time()
print("distance grid computation time: %1.3fs"%(end-strt))

strt = time.time()
pld  = pore.get_pld(S_dgrid)
end  = time.time()
print("PLD: %1.3f" % pld)
print("PLD computation time: %1.3fs"%(end-strt))

strt = time.time()
lcd  = pore.get_lcd(S_dgrid)
end  = time.time()
print("LCD: %1.3f" % lcd)
print("LCD computation time: %1.3fs"%(end-strt))

### For PLD in a different direction

In [ ]:
%%time
pld = pipe(S, 
           lambda s: np.rot90(s, axes=(0,2)),
           lambda s: np.pad(s, padval, 'constant', constant_values=0),
           lambda s: pore.dgrid(s, len_pixel=len_pixel),
           lambda s: pore.get_pld(s))
print(pld)

# Shortest Path Algorithm

- Convert admissible coordinates to graph structure
 - generate adjacency matrix from graph structure
 - use adjacency matrix to identify shortest paths
 - Use DFS or BFS or Shortest Path on adjacency matrix to prune the medial path and scan through all independant medial paths to identify PLD
 - For the pruned path, use eculidean distance from nearest atoms as an additional metric and try to identify the flow through the structure as a heuristic.

In [ ]:
strt = time.time()
S_1 = (pore.gen_cleanPore(S_dgrid, r_probe=1.0, r_min=2.5, len_pixel=len_pixel) > 0) * 1
end = time.time()
print("Pore Cleaning Computation Time: %1.3fs" % (end-strt))

strt = time.time()
S_11 = np.pad(S_1, pad_width=((0,0),(0,0),(len_pixel, len_pixel)), mode = "constant", constant_values=1)
S_2 = pore.gen_medialAxis(S_11)[:,:,len_pixel:-len_pixel]
end = time.time()
print("Medial Path Computation Time: %1.3fs" % (end-strt))

strt = time.time()
S_3, paths = pore.gen_throughPath(S_2, depth=1)
end = time.time()
print("Through Path Computation Time: %1.3fs" % (end-strt))
print("Mean and setdev of path lengths: %1.3f, %1.3f" % (np.mean(paths), np.std(paths)))

n_paths = len(pore.return_labelled(S_1)[-1])
print("No. of unique paths: %d" % n_paths)
      
asa = pore.get_asa(S_1, len_pixel=10)
print("Probe Accessible Surface Area: %1.3f" % asa)
av = np.count_nonzero(S_1) * (1 / len_pixel)**3
print("Probe Accessible Volume: %1.3f" % av)

psd = S_dgrid[S_2==1]
print("Mean and setdev of pore size distribution: %1.3f, %1.3f" % (np.mean(psd), np.std(psd)))

dim = np.asarray(S.shape) / len_pixel
print("dimensions of the structure: ", dim)

In [ ]:
plt.hist(paths)
plt.show()

In [ ]:
%%time
write2vtk(S_1, "%s_pore.vtk" % cif)
write2vtk(S_2, "%s_path.vtk" % cif)
write2vtk(S_3, "%s_through_path.vtk" % cif)